In [8]:
from copy import deepcopy

import numpy as np
from sklearn.metrics import roc_auc_score 
import tqdm

from rocauc_pairwise.deltaauc_cpu import deltaauc_exact
from rocauc_pairwise.deltaauc_cpu import deltaauc
from rocauc_pairwise.utils import get_labelscount_borders

In [2]:
def delta_auc_score(y_true, y_pred, i, j):
    auc_1 = roc_auc_score(y_true, y_pred)
    y_pred_ = deepcopy(y_pred)
    y_pred_[i], y_pred_[j] = y_pred_[j], y_pred_[i]
    auc_2 = roc_auc_score(y_true, y_pred_)
    return auc_1 - auc_2   

# Simple test

In [3]:
y_true = [0, 1, 0, 1, 0, 1, 1, 0, 1, 0]
y_pred = [0.5, 0.5, 0.5, 0.1, 0.05, 0.02, 0.02, 0.02, 0.02, 0.02]
y_pred = np.array(y_pred)
y_true = np.array(y_true)

In [4]:
n_ones, n_zeroes = np.sum(y_true), len(y_true) - np.sum(y_true)
y_pred_argsorted = np.argsort(y_pred)[::-1]
counters_p, counters_n, y_pred_left, y_pred_right = get_labelscount_borders(y_true, y_pred, y_pred_argsorted)

In [5]:
deltaauc_true = delta_auc_score(y_true, y_pred, 0, 6)
deltaauc_numba = deltaauc_exact(y_true, y_pred, 
                                counters_p, counters_n,
                                y_pred_left, y_pred_right, 
                                n_ones, n_zeroes,
                                0, 6)

In [6]:
assert(np.abs(deltaauc_true - deltaauc_numba) < 1e-6)

# Random tests 

In [7]:
def compute_deltaauc_exact(y_true, y_pred, i, j):
    n_ones, n_zeroes = np.sum(y_true), len(y_true) - np.sum(y_true)
    y_pred_argsorted = np.argsort(y_pred)[::-1]
    counters_p, counters_n, y_pred_left, y_pred_right = get_labelscount_borders(y_true, y_pred, y_pred_argsorted)
    
    deltaauc_numba = deltaauc_exact(y_true, y_pred, 
                                counters_p, counters_n,
                                y_pred_left, y_pred_right, 
                                n_ones, n_zeroes,
                                i, j)
    return deltaauc_numba

def get_random_preds(n_unique, n):
    y_true = np.random.randint(0, 2, n)
    y_pred = np.random.randint(0, n_unique, n)
    if np.sum(y_true) == len(y_true):
        y_true[0] = 0
    if np.sum(y_true) == 0:
        y_true[0] = 1
    return y_true, y_pred

In [9]:
aucs = []

for k in tqdm.tqdm(range(10000)):
    y_true, y_pred = get_random_preds(7, 10)
    for s in range(30):
        i, j = np.random.randint(0, 10), np.random.randint(0, 10)
        tmp = delta_auc_score(y_true, y_pred, i, j)
        tmp1 = compute_deltaauc_exact(y_true, y_pred, i, j)
        aucs.append(tmp1)
        assert np.abs(tmp - tmp1) < 1e-4

100%|██████████| 10000/10000 [05:16<00:00, 31.61it/s]
